In [1]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import sys
import warnings
warnings.filterwarnings('ignore')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import sqlite3
import pandas_profiling
#from common_function import *

pd.options.display.max_columns = 999

In [7]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
    

def reduce_mem_usage(df_):
    start_mem = df_.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for c in df_.columns[df_.dtypes != 'object']:
        col_type = df_[c].dtype
        
        c_min = df_[c].min()
        c_max = df_[c].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df_[c] = df_[c].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df_[c] = df_[c].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df_[c] = df_[c].astype(np.int32)
            else:
                df_[c] = df_[c].astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df_[c] = df_[c].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df_[c] = df_[c].astype(np.float32)
            else:
                df_[c] = df_[c].astype(np.float64)

    end_mem = df_.memory_usage().sum() / 1024**2
    print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df_

In [134]:
# Load the csv 
df_match_wide = pd.read_csv("data/ATP_matches.csv")

# Store the shape of the data for reference check later
original_shape = df_match_wide.shape

In [135]:
# What are the big tournaments in the data
df_match_wide.Tournament.value_counts().head()

Australian Open, Melbourne    1665
French Open, Paris            1664
U.S. Open, New York           1662
Wimbledon, London             1106
Indian Wells                   653
Name: Tournament, dtype: int64

In [136]:
missing_values_table(df_match_wide)

Your selected dataframe has 35 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
Round_Description,34,0.2


In [137]:
df_match_wide.head(2)

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,Winner_Games_Won,Winner_Aces,Winner_DoubleFaults,Winner_FirstServes_Won,Winner_FirstServes_In,Winner_SecondServes_Won,Winner_SecondServes_In,Winner_BreakPoints_Won,Winner_BreakPoints,Winner_ReturnPoints_Won,Winner_ReturnPoints_Faced,Winner_TotalPoints_Won,Loser_Sets_Won,Loser_Games_Won,Loser_Aces,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106,97,0,2,12,5,2,22,30,12,19,4,7,25,59,59,0,7,2,3,21,33,13,26,1,3,15,49,49
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83,48,0,2,12,2,0,14,17,11,16,6,14,36,58,61,0,1,1,4,17,32,5,26,0,1,8,33,30


In [138]:
df_match_wide.Tournament_Date = pd.to_datetime(df_match_wide.Tournament_Date,dayfirst=True)

In [139]:
df_match_wide['Total_Games'] = df_match_wide.Winner_Games_Won + df_match_wide.Loser_Games_Won

In [140]:
winner_cols = ['Winner', 'Tournament', 'Tournament_Date', 'Court_Surface',
       'Round_Description', 'Winner_Rank', 'Retirement_Ind',
       'Winner_Sets_Won', 'Winner_Games_Won', 'Winner_Aces',
       'Winner_DoubleFaults', 'Winner_FirstServes_Won',
       'Winner_FirstServes_In', 'Winner_SecondServes_Won',
       'Winner_SecondServes_In', 'Winner_BreakPoints_Won',
       'Winner_BreakPoints', 'Winner_ReturnPoints_Won',
       'Winner_ReturnPoints_Faced', 'Winner_TotalPoints_Won','Total_Games']
loser_cols = ['Loser', 'Tournament', 'Tournament_Date', 'Court_Surface',
       'Round_Description', 'Loser_Rank', 'Retirement_Ind',
        'Loser_Sets_Won','Loser_Games_Won', 'Loser_Aces', 'Loser_DoubleFaults',
       'Loser_FirstServes_Won', 'Loser_FirstServes_In',
       'Loser_SecondServes_Won', 'Loser_SecondServes_In',
       'Loser_BreakPoints_Won', 'Loser_BreakPoints', 'Loser_ReturnPoints_Won',
       'Loser_ReturnPoints_Faced', 'Loser_TotalPoints_Won','Total_Games']
df_winner = df_match_wide[winner_cols]

df_loser = df_match_wide[loser_cols]

df_winner["won"] = 1
df_loser["won"] = 0

# Rename the columns so that they are the same for winners and losers
df_winner.columns = ['Player', 'Tournament', 'Tournament_Date', 'Court_Surface',
       'Round_Description', 'Rank', 'Retirement_Ind',
        'Sets_Won','Games_Won', 'Aces', 'DoubleFaults',
       'FirstServes_Won', 'FirstServes_In',
       'SecondServes_Won', 'SecondServes_In',
       'BreakPoints_Won', 'BreakPoints', 'ReturnPoints_Won',
       'ReturnPoints_Faced', 'TotalPoints_Won',"won",'Total_Games']

# Rename the columns so that they are the same for winners and losers
df_loser.columns = df_winner.columns

In [141]:
# append the winner and loser dfs together so now the dataframe is in long format

df_match_long = df_winner.append(df_loser)

In [142]:
# The long table should have exactly twice of the rows of the original data
assert df_match_long.shape[0] == original_shape[0]*2

In [143]:
# Get the dates of the aus open
aus_open_dates = df_match_wide.loc[df_match_wide.Tournament=='Australian Open, Melbourne'].groupby(['Tournament','Tournament_Date'])['Winner'].count().reset_index()

aus_open_dates

,Tournament,Tournament_Date,Winner
0,"Australian Open, Melbourne",2012-01-16,237
1,"Australian Open, Melbourne",2013-01-14,239
2,"Australian Open, Melbourne",2014-01-13,238
3,"Australian Open, Melbourne",2015-01-19,238
4,"Australian Open, Melbourne",2016-01-18,239
5,"Australian Open, Melbourne",2017-01-16,239
6,"Australian Open, Melbourne",2018-01-15,235


In [144]:
# 'Australian Open, Melbourne'    
# # 'French Open, Paris'            
# # 'U.S. Open, New York '          
# # 'Wimbledon, London'           

In [145]:
# some cells have . but we will convert them into null
df_match_long = df_match_long.replace('.', np.nan)

stats_cols = ['Rank', 'Retirement_Ind', 'Sets_Won', 'Games_Won',
       'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In',
       'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints',
       'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won',]

df_match_long[stats_cols] = df_match_long[stats_cols].apply(pd.to_numeric)


df_match_long = df_match_long.sort_values(['Player','Tournament_Date','Tournament'], ascending=True)

In [146]:
# df_player = get_feature_period(df_player)

In [147]:

df_match_long['Total_Games'] = df_match_long.FirstServes_Won
df_match_long['F_Player_Serve_Win_Ratio'] = (df_match_long.FirstServes_Won + df_match_long.SecondServes_Won - df_match_long.DoubleFaults)/(df_match_long.FirstServes_In + df_match_long.SecondServes_In + df_match_long.DoubleFaults)# Point Win ratio when serving
df_match_long['F_Player_Return_Win_Ratio'] = df_match_long.ReturnPoints_Won / df_match_long.ReturnPoints_Faced # Point win ratio when returning
df_match_long['F_Player_BreakPoints_Per_Return_Game'] = df_match_long.BreakPoints/df_match_long.Total_Games  # Breakpoints per receiving game
df_match_long['F_Player_Game_Win_Percentage'] = df_match_long.Games_Won/df_match_long.Total_Games

In [148]:
df_match_long.columns

Index(['Player', 'Tournament', 'Tournament_Date', 'Court_Surface',
       'Round_Description', 'Rank', 'Retirement_Ind', 'Sets_Won', 'Games_Won',
       'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In',
       'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints',
       'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won',
       'Total_Games', 'F_Player_Serve_Win_Ratio', 'F_Player_Return_Win_Ratio',
       'F_Player_BreakPoints_Per_Return_Game', 'F_Player_Game_Win_Percentage'],
      dtype='object')

In [169]:
def get_rolling_features (df):
    
    # Need to sort the data first so its from old to recent
    df = df.sort_values(['Player','Tournament_Date','Tournament'], ascending=True)
    
    
    # columns we are taking rolling averages from
    num_cols = ['Rank', 'Retirement_Ind', 'Sets_Won', 'Games_Won',
       'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In',
       'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints',
       'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won','F_Player_Serve_Win_Ratio',
       'F_Player_Return_Win_Ratio', 'F_Player_BreakPoints_Per_Return_Game',
       'F_Player_Game_Win_Percentage']
    
    # For each year, get the rolling averages of that player's past matches before the tournament start date
    for index, tournament_date in enumerate(aus_open_dates.Tournament_Date):
    
        # create a temp df to store the intrim results
        df_temp = df.loc[df.Tournament_Date < tournament_date]

        # take the most recent 20 matches for the rolling average
        df_temp = df_temp.groupby('Player')[num_cols].rolling(20, min_periods=1).mean().reset_index()
        df_temp = df_temp.groupby('Player').tail(1)

        if index ==0:
            # create the result dataframe
            df_result = df_temp
            # so we know which tournament this feature is for
            df_result['tournament_date_index'] = tournament_date
        else:
            df_temp['tournament_date_index'] = tournament_date
            df_result = df_result.append(df_temp)
    df_result.drop('level_1', axis=1,inplace=True)
    
    return df_result

In [170]:
# df_feature = pd.DataFrame()

# df_player = df_player.copy()

# num_cols = ['Rank', 'Retirement_Ind', 'Sets_Won', 'Games_Won',
#        'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In',
#        'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints',
#        'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won','F_Player_Serve_Win_Ratio',
#        'F_Player_Return_Win_Ratio', 'F_Player_BreakPoints_Per_Return_Game',
#        'F_Player_Game_Win_Percentage']

# for index, year in enumerate(aus_open_dates.Tournament_Date):
    
#     df_year = df_test.loc[df_test.Tournament_Date < year]
    
#     df_year = df_year.groupby('Player')[num_cols].rolling(20, min_periods=1).mean().reset_index()
#     df_year = df_year.groupby('Player').tail(1)
#     #df_year = df_year.groupby('Player')[num_cols].mean().reset_index()
#     if index ==0:
#         df_new = df_year
#         df_new['year'] = year
#     else:
#         df_year['year'] = year
#         df_new = df_new.append(df_year)
        
    
    
df_result_long = get_rolling_features (df_match_long)
    

In [171]:
df_match_long.shape

(41614, 26)

In [172]:
#df_match_long = df_match_long.sort_values(['Player','Tournament_Date'])

In [173]:
# check some of the matches


In [174]:
# df_match_open = df_match.loc[df_match.Tournament=='Australian Open, Melbourne',['Winner','Loser','Tournament_Date']] 

# df_wide = df_match_open.merge(df_new, left_on=['Winner','Tournament_Date'], right_on = ['Player','year'],how='left',validate ='m:1')

In [175]:
df_match_long.head(20)

,Player,Tournament,Tournament_Date,Court_Surface,Round_Description,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,Total_Games,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage
3121,Adam Feeney,"Australian Open, Melbourne",2013-01-14,Hard,Qualifying,327.0,0,2.0,17,0.0,8.0,42.0,63.0,19.0,33.0,4.0,19.0,48.0,115.0,109,31,42.0,0.509615,0.417391,0.452381,0.404762
3173,Adam Feeney,"Australian Open, Melbourne",2013-01-14,Hard,Qualifying,327.0,0,0.0,9,1.0,2.0,33.0,54.0,7.0,24.0,3.0,7.0,29.0,75.0,69,22,33.0,0.475000,0.386667,0.212121,0.272727
8917,Adam Pavlasek,"Australian Open, Melbourne",2015-01-19,Hard,Qualifying,240.0,0,2.0,13,5.0,1.0,28.0,37.0,15.0,25.0,3.0,7.0,NaN,NaN,71,20,28.0,0.666667,NaN,0.250000,0.464286
8978,Adam Pavlasek,"Australian Open, Melbourne",2015-01-19,Hard,Qualifying,240.0,0,1.0,11,18.0,10.0,66.0,95.0,38.0,78.0,2.0,5.0,NaN,NaN,147,27,66.0,0.513661,NaN,0.075758,0.166667
11062,Adam Pavlasek,"U.S. Open, New York",2015-08-31,Hard,Qualifying,149.0,0,0.0,7,2.0,1.0,17.0,31.0,17.0,30.0,1.0,1.0,NaN,NaN,52,20,17.0,0.532258,NaN,0.058824,0.411765
13080,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,2.0,15,9.0,3.0,42.0,66.0,15.0,30.0,5.0,11.0,40.0,88.0,97,28,42.0,0.545455,0.454545,0.261905,0.357143
13131,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,2.0,12,3.0,0.0,23.0,27.0,11.0,16.0,4.0,6.0,23.0,47.0,57,16,23.0,0.790698,0.489362,0.260870,0.521739
13214,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,First Round,132.0,0,3.0,23,7.0,2.0,66.0,102.0,26.0,53.0,8.0,17.0,61.0,135.0,153,41,66.0,0.573248,0.451852,0.257576,0.348485
13149,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,1.0,10,2.0,2.0,33.0,51.0,11.0,31.0,2.0,4.0,26.0,71.0,70,25,33.0,0.500000,0.366197,0.121212,0.303030
13229,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Second Round,132.0,0,0.0,10,5.0,4.0,42.0,55.0,12.0,29.0,0.0,0.0,11.0,67.0,65,28,42.0,0.568182,0.164179,0.000000,0.238095


In [176]:
example_true = df_match_long.loc[df_match_long.Player == 'Adam Pavlasek']

In [177]:
example_calculated = df_result_long.loc[df_result_long.Player == 'Adam Pavlasek']

In [178]:
example_true

,Player,Tournament,Tournament_Date,Court_Surface,Round_Description,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,Total_Games,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage
8917,Adam Pavlasek,"Australian Open, Melbourne",2015-01-19,Hard,Qualifying,240.0,0,2.0,13,5.0,1.0,28.0,37.0,15.0,25.0,3.0,7.0,NaN,NaN,71,20,28.0,0.666667,NaN,0.250000,0.464286
8978,Adam Pavlasek,"Australian Open, Melbourne",2015-01-19,Hard,Qualifying,240.0,0,1.0,11,18.0,10.0,66.0,95.0,38.0,78.0,2.0,5.0,NaN,NaN,147,27,66.0,0.513661,NaN,0.075758,0.166667
11062,Adam Pavlasek,"U.S. Open, New York",2015-08-31,Hard,Qualifying,149.0,0,0.0,7,2.0,1.0,17.0,31.0,17.0,30.0,1.0,1.0,NaN,NaN,52,20,17.0,0.532258,NaN,0.058824,0.411765
13080,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,2.0,15,9.0,3.0,42.0,66.0,15.0,30.0,5.0,11.0,40.0,88.0,97,28,42.0,0.545455,0.454545,0.261905,0.357143
13131,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,2.0,12,3.0,0.0,23.0,27.0,11.0,16.0,4.0,6.0,23.0,47.0,57,16,23.0,0.790698,0.489362,0.260870,0.521739
13214,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,First Round,132.0,0,3.0,23,7.0,2.0,66.0,102.0,26.0,53.0,8.0,17.0,61.0,135.0,153,41,66.0,0.573248,0.451852,0.257576,0.348485
13149,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Qualifying,132.0,0,1.0,10,2.0,2.0,33.0,51.0,11.0,31.0,2.0,4.0,26.0,71.0,70,25,33.0,0.500000,0.366197,0.121212,0.303030
13229,Adam Pavlasek,"French Open, Paris",2016-05-23,Clay,Second Round,132.0,0,0.0,10,5.0,4.0,42.0,55.0,12.0,29.0,0.0,0.0,11.0,67.0,65,28,42.0,0.568182,0.164179,0.000000,0.238095
13717,Adam Pavlasek,Kitzbuhel,2016-07-18,Clay,First Round,116.0,0,2.0,12,3.0,4.0,25.0,30.0,20.0,34.0,2.0,6.0,25.0,60.0,70,19,25.0,0.602941,0.416667,0.240000,0.480000
13725,Adam Pavlasek,Kitzbuhel,2016-07-18,Clay,Second Round,116.0,0,2.0,12,6.0,2.0,24.0,32.0,14.0,25.0,4.0,5.0,21.0,45.0,59,18,24.0,0.610169,0.466667,0.208333,0.500000


In [179]:
example_calculated

,Player,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage,tournament_date_index
4,Adam Pavlasek,209.666667,0.0,1.000000,10.333333,8.333333,4.000000,37.000000,54.333333,23.333333,44.333333,2.000000,4.333333,NaN,NaN,90.000000,22.333333,0.570862,NaN,0.128194,0.347572,2016-01-18
13,Adam Pavlasek,143.833333,0.0,1.333333,12.416667,6.250000,3.083333,36.666667,52.833333,17.833333,35.416667,2.833333,5.833333,29.00,76.0000,83.666667,24.750000,0.580659,0.382870,0.165485,0.368552,2017-01-16
20,Adam Pavlasek,129.894737,0.0,1.157895,11.842105,6.578947,3.368421,34.947368,50.842105,16.736842,35.210526,2.789474,6.105263,28.75,77.6875,80.578947,25.263158,0.544621,0.364698,0.173089,0.351148,2018-01-15


In [180]:
example_true.loc[example_true.Tournament_Date<'2018-01-15','Rank'].mean()

129.89473684210526

In [181]:
example_true.shape

(28, 26)

In [182]:
example_true.loc[example_true.Tournament_Date<'2016-01-18','Rank'].mean()

209.66666666666666

In [206]:
df_result_long.tail()

,Player,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage,tournament_date_index
35761,Yusuke Takahashi,262.000000,0.0,1.00,14.000000,4.00,2.000000,28.0,43.00,22.000000,44.000000,4.000000,10.000000,36.000000,90.000000,86.000000,30.00,0.539326,0.400000,0.357143,0.500000,2018-01-15
35762,Zbynek Mlynarik,238.000000,0.0,1.00,12.000000,9.00,4.000000,32.0,46.00,20.000000,50.000000,3.000000,7.000000,28.000000,80.000000,80.000000,28.00,0.480000,0.350000,0.218750,0.375000,2018-01-15
35796,Ze Zhang,208.200000,0.0,0.75,9.850000,3.85,3.800000,29.8,46.95,11.850000,26.100000,2.350000,5.850000,23.529412,66.823529,67.450000,22.85,0.490419,0.346640,0.190623,0.321596,2018-01-15
35802,Zhe Li,258.666667,0.0,0.50,6.666667,2.50,0.833333,23.0,37.50,9.166667,21.333333,1.166667,3.666667,17.500000,54.833333,49.666667,18.50,0.503743,0.302598,0.150431,0.277606,2018-01-15
35807,Zhizhen Zhang,530.200000,0.0,1.40,11.600000,5.80,2.600000,30.6,42.80,15.800000,32.800000,2.400000,6.000000,26.600000,71.600000,73.000000,22.60,0.552725,0.366874,0.223653,0.393075,2018-01-15


In [195]:
# Randomise the match_wide dataset so the first player is not always the winner

# set a seed so the random number is reproducable
np.random.seed(2)

# randomise a number 0/1 with 50% chance each
# if 0 then take the winner, 1 then take loser

df_match_wide['random_number'] = np.random.randint(2, size=len(df_match_wide))

df_match_wide['randomised_player'] = np.where(df_match_wide['random_number']==0,df_match_wide['Winner'],df_match_wide['Loser'])

# set the target (win/loss) based on the new randomise number

df_match_wide['target'] = np.where(df_match_wide['random_number']==0,1,0)

In [225]:
df_match_wide.tail()

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,Winner_Games_Won,Winner_Aces,Winner_DoubleFaults,Winner_FirstServes_Won,Winner_FirstServes_In,Winner_SecondServes_Won,Winner_SecondServes_In,Winner_BreakPoints_Won,Winner_BreakPoints,Winner_ReturnPoints_Won,Winner_ReturnPoints_Faced,Winner_TotalPoints_Won,Loser_Sets_Won,Loser_Games_Won,Loser_Aces,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won,Total_Games,random_number,randomised_player,target
20802,Karen Khachanov,Novak Djokovic,Paris,2018-10-29,Indoor Hard,Finals,18,2,0,2,13,9,2,38,50,8,16,3,8,27,71,73,0,9,5,1,30,43,14,28,1,5,20,66,64,22,0,Karen Khachanov,1
20803,Jaume Antoni Munar Clar,Frances Tiafoe,Milan,2018-11-05,Indoor Hard,NaN,76,40,0,3,12,7,1,25,27,16,19,2,4,19,46,60,0,5,6,3,21,29,6,17,0,2,5,46,32,17,0,Jaume Antoni Munar Clar,1
20804,Frances Tiafoe,Hubert Hurkacz,Milan,2018-11-05,Indoor Hard,NaN,40,85,0,3,14,11,1,39,53,17,25,2,7,22,67,78,1,10,9,4,35,48,10,19,1,7,22,78,67,24,1,Hubert Hurkacz,0
20805,Hubert Hurkacz,Jaume Antoni Munar Clar,Milan,2018-11-05,Indoor Hard,NaN,85,76,0,3,17,5,3,38,53,13,27,6,12,40,98,91,2,13,8,1,43,63,15,35,3,9,29,80,87,30,0,Hubert Hurkacz,1
20806,Andrey Rublev,Liam Caruana,Milan,2018-11-05,Indoor Hard,NaN,68,.,0,3,12,9,2,33,39,6,18,3,5,21,53,60,0,6,6,1,28,39,4,14,1,3,18,57,50,18,1,Liam Caruana,0


In [194]:
df_match_wide['random_number'].mean()

0.5032441005430864

In [197]:
df_match_wide['target'].mean()

0.49675589945691356

In [226]:
df_match_wide_aus_open = df_match_wide.loc[df_match_wide.Tournament=='Australian Open, Melbourne', \
                                           ['Winner','Loser','Tournament','Tournament_Date',
                                           'target','randomised_player','random_number']]

# Get the rolling features

df_match_wide_aus_open_features = df_match_wide_aus_open.merge(df_result_long, how='left',
                         left_on = ['randomised_player','Tournament_Date'],
                         right_on = ['Player','tournament_date_index'],validate ='m:1')

In [227]:
df_match_wide_aus_open_features.loc[df_match_wide_aus_open_features.Player.isna()].shape

(196, 29)

In [228]:
df_match_wide_aus_open_features.loc[df_match_wide_aus_open_features.Player.isna(),'Tournament_Date'].value_counts()

2012-01-16    128
2014-01-13     22
2013-01-14     19
2018-01-15     10
2015-01-19      7
2017-01-16      5
2016-01-18      5
Name: Tournament_Date, dtype: int64

In [229]:
df_match_wide_aus_open_features.loc[
    (df_match_wide_aus_open_features.Player.isna())
    & (df_match_wide_aus_open_features.Tournament_Date=='2018-01-15')
].head()

,Winner,Loser,Tournament,Tournament_Date,target,randomised_player,random_number,Player,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage,tournament_date_index
1436,Stefan Kozlov,Blake Ellis,"Australian Open, Melbourne",2018-01-15,0,Blake Ellis,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1441,Di Wu,Dayne Kelly,"Australian Open, Melbourne",2018-01-15,0,Dayne Kelly,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1444,Bjorn Propst,Facundo Bagnis,"Australian Open, Melbourne",2018-01-15,1,Bjorn Propst,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1453,Carlos Taberner,Joao Monteiro,"Australian Open, Melbourne",2018-01-15,1,Carlos Taberner,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1462,Karim-Mohamed Maamoun,Marcelo Arevalo,"Australian Open, Melbourne",2018-01-15,1,Karim-Mohamed Maamoun,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [221]:
df_match_long.loc[df_match_long.Player=='Dayne Kelly']

,Player,Tournament,Tournament_Date,Court_Surface,Round_Description,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,Total_Games,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage
17915,Dayne Kelly,"Australian Open, Melbourne",2018-01-15,Hard,Qualifying,274.0,0,0.0,7,1.0,5.0,23.0,41.0,12.0,31.0,2.0,4.0,24.0,66.0,59,20,23.0,0.38961,0.363636,0.173913,0.304348


In [222]:
df_result_long.loc[df_result_long.Player=='Dayne Kelly']

,Player,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage,tournament_date_index


In [231]:
df_match_wide_aus_open_features.tail()

,Winner,Loser,Tournament,Tournament_Date,target,randomised_player,random_number,Player,Rank,Retirement_Ind,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won,F_Player_Serve_Win_Ratio,F_Player_Return_Win_Ratio,F_Player_BreakPoints_Per_Return_Game,F_Player_Game_Win_Percentage,tournament_date_index
1660,Hyeon Chung,Tennys Sandgren,"Australian Open, Melbourne",2018-01-15,1,Hyeon Chung,0,Hyeon Chung,55.40,0.05,1.45,12.0,4.20,3.3,32.60,46.25,13.80,28.05,2.65,6.50,28.60,74.25,75.0,23.2,0.552529,0.385395,0.214333,0.375871,2018-01-15
1661,Roger Federer,Tomas Berdych,"Australian Open, Melbourne",2018-01-15,1,Roger Federer,0,Roger Federer,2.40,0.00,2.00,15.4,8.95,2.0,35.30,44.40,17.65,29.65,3.55,8.85,35.35,88.30,88.3,26.4,0.684348,0.402656,0.268957,0.449503,2018-01-15
1662,Roger Federer,Hyeon Chung,"Australian Open, Melbourne",2018-01-15,0,Hyeon Chung,1,Hyeon Chung,55.40,0.05,1.45,12.0,4.20,3.3,32.60,46.25,13.80,28.05,2.65,6.50,28.60,74.25,75.0,23.2,0.552529,0.385395,0.214333,0.375871,2018-01-15
1663,Marin Cilic,Kyle Edmund,"Australian Open, Melbourne",2018-01-15,1,Marin Cilic,0,Marin Cilic,5.05,0.05,1.45,12.5,10.00,2.8,32.85,42.00,14.25,29.55,2.90,6.45,27.90,71.65,75.0,23.2,0.635077,0.393866,0.205465,0.401909,2018-01-15
1664,Roger Federer,Marin Cilic,"Australian Open, Melbourne",2018-01-15,0,Marin Cilic,1,Marin Cilic,5.05,0.05,1.45,12.5,10.00,2.8,32.85,42.00,14.25,29.55,2.90,6.45,27.90,71.65,75.0,23.2,0.635077,0.393866,0.205465,0.401909,2018-01-15
